# 唯一计数器

如果要统计访问网站的用户数量而不是网站被浏览的次数，那么上一章提到的计数器是无法实现的，因为它无法判断访问网站的是不同的用户还是重复访问网站多次的同一个用户。而唯一计数器的目的就是来实现这种场景，只会对不同的用户进行计数。

有两种方案：
+ 使用集合键
+ 使用 HyperLogLog 键

### 使用集合键

比如要统计访问网站的用户数量，只需要一直使用 `SADD` 命令将用户添加到集合中即可，集合只会保留不重复的元素。之后，只需 `SCARD` 命令就可以获取当前访问网站的用户数量。而且还可以使用 `SREM` 从集合中移除指定的用户。

In [3]:
class UniqueCounter:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def include(self, item):
        """
        尝试对给定元素进行计数。
        如果该元素之前没有被计数过，那么返回True，否则返回False。
        """
        # SADD key "Tom"
        return self.client.sadd(self.key, item) == 1

    def exclude(self, item):
        """
        尝试将被计数的元素移出计数器。
        移除成功返回True，因元素尚未被计数而导致移除失败则返回False。
        """
        # SREM key "Tom"
        return self.client.srem(self.key, item) == 1

    def count(self):
        """
        返回计数器当前已计数的元素数量。
        如果计数器为空，那么返回0作为结果。
        """
        # SCARD key
        return self.client.scard(self.key)

### 使用 HyperLogLog 键

当需要计数的元素数量非常多，使用集合键的方式将消耗大量内存。使用 HyperLogLog 键可以解决这个问题。

HyperLogLog 和集合的相同与不同之处如下：
+ HyperLogLog 和集合一样，都可以对元素进行计数。
+ HyperLogLog 和集合的不同之处在于，它返回的计数结果并**不是准确的集合基数**，而是一个与基数八九不离十的**估算基数**。（基数就是说集合中不重复的元素个数）
+ HyperLogLog 的好处是它的占用量不会随着被计数元素增多而增多，**无论对多少元素进行计数，HyperLogLog 的内存开销都是固定的，并且是非常少的**。

如果并不追求完全正确的计数结果，并且不需要准确知道某个元素是否已经被计数，那么完全可以使用 HyperLogLog 代替集合来实现唯一计数器。另外 HyperLogLog 无法撤销已添加的元素。

关于 HyperLogLog 的实现，比较复杂，比较数学，不去研究了。[《快速估算万亿集合中不重复元素的数量：HyperLogLog算法》](https://www.bilibili.com/video/BV1Nw4m1o72Z/)这个视频可以快速了解它是个啥，[《HyperLogLog》](https://www.yuque.com/abser/blog/mrv5ke)介绍了实现的细节。

简单概括一下，HyperLogLog 是 Redis 中的一种概率数据结构，用于以极低的内存开销（约12KB）估算数据集的唯一元素数量，误差约为0.81%。它通过对元素进行哈希，统计哈希值二进制中最大前导零的个数，并利用统计方法估算基数。非常适合大规模场景，比如统计网站唯一访客，相比集合更节省内存，但结果是近似的。

HyperLogLog 的过程：
> 1.哈希元素：  
每个用户 ID（例如 "user123"）通过哈希函数（如 Redis 使用的 murmurhash）转换为一个 64 位整数。  
假设 "user123" 哈希后得到二进制值：0000101100...（前 4 位是零）。  
2.统计前导零：  
检查哈希值的二进制表示，从最高位开始数连续的零（前导零）。  
对于 0000101100...，前导零个数是 4（因为前 4 位是 0，第五位是 1）。  
这个前导零个数反映了哈希值的“稀有性”：前导零越多，值越“稀有”，暗示数据集的基数可能越大。  
3.分配到桶：  
Redis 的 HyperLogLog 默认使用 16,384 个桶（2^14）。  
哈希值的前几位（例如前 14 位）决定这个值分配到哪个桶。  
假设 "user123" 的哈希值分配到桶 #100，桶 #100 记录下当前最大前导零数（比如 4）。  
如果后续另一个用户（如 "user456"）的哈希值也分配到桶 #100，且前导零数为 6，则更新桶 #100 的值为 6（只保留最大值）。  
4.处理多个元素：  
对每个用户 ID 重复上述步骤：哈希 → 统计前导零 → 更新对应桶的最大前导零数。  
例如，"user456" 哈希后可能分配到桶 #200，记录前导零数 3；"user789" 分配到桶 #100，记录前导零数 5（桶 #100 仍保留 6，因为 6 > 5）。  
5.估算基数：  
遍历所有 16,384 个桶，获取每个桶记录的最大前导零数。  
使用调和平均数（harmonic mean）计算这些值的平均稀有性，结合数学公式估算总的唯一元素数量。  
Redis 还应用了偏差校正：
当基数很小（接近 0）或很大（接近 2^32）时，使用修正公式提高准确性。
对于中等基数，估算公式基于 $\alpha \cdot m^2 / \sum (2^{-M_i})$，其中 m 是桶数，Mi 是每个桶的最大前导零数，α 是常数。
6.结果：  
假设处理了 10 万个用户 ID，HyperLogLog 估算出大约 99,500 个唯一用户，实际可能有 100,000 个，误差在 0.81% 左右。  
整个过程只用约 12KB 内存（16,384 桶 × 6 位/桶），远小于存储所有用户 ID 的集合。  

In [5]:
class HllUniqueCounter:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def include(self, item):
        """
        尝试对给定元素进行计数。
        如果该元素之前没有被计数过，那么返回True，否则返回False。
        """
        # PFDD key “Tom”
        return self.client.pfadd(self.key, item) == 1

    def exclude(self, item):
        """
        尝试将被计数的元素移出计数器。
        移除成功返回True，因元素尚未被计数而导致移除失败则返回False。
        """
        raise NotImplementedError

    def count(self):
        """
        返回计数器当前已计数的元素数量。
        如果计数器为空，那么返回0作为结果。
        """
        # PFCOUNT key
        return self.client.pfcount(self.key)

另外在看上面提到的呢个视频时，评论区有人问，布隆过滤器可以吗？每次判断这个元素子在不在布隆过滤器中，没有的话计数器加 1，并把这个元素映射到布隆过滤器中。

网友的回复：
+ 千万的数据还是挺准确的，过亿的数据就误差有点大，可能是长度开得不够大，而且过亿的数据计算时间有点长。
+ 布隆需要共享存储，没法彻底分布式运算。（HyperLogLog 这个算法可以，大概就是对不同服务器上的桶取并集，然后数学计算）
+ 布隆过滤器的误报概率会随着数据量的增加而增大。